In [77]:
# Boilerplate
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

import matplotlib
%matplotlib inline
from matplotlib.pyplot import *

# List physical devices
tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [119]:
from importlib import reload

import backbone
import rpn

if 'data_utils' not in sys.modules:
    import data_utils
    import backbone
    import rpn
else:
    reload(data_utils)
    reload(backbone)
    reload(rpn)

In [120]:
# Instantiate the backbone
test = backbone.Backbone_InceptionResNetV2()
test.extractor.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 720, 1280,   0           []                               
                                3)]                                                               
                                                                                                  
 conv2d_1624 (Conv2D)           (None, 359, 639, 32  864         ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_1624 (Batc  (None, 359, 639, 32  96         ['conv2d_1624[0][0]']            
 hNormalization)                )                                               

                                                                  'activation_1755[0][0]']        
                                                                                                  
 block17_14_conv (Conv2D)       (None, 43, 78, 1088  418880      ['block17_14_mixed[0][0]']       
                                )                                                                 
                                                                                                  
 block17_14 (Lambda)            (None, 43, 78, 1088  0           ['block17_13_ac[0][0]',          
                                )                                 'block17_14_conv[0][0]']        
                                                                                                  
 block17_14_ac (Activation)     (None, 43, 78, 1088  0           ['block17_14[0][0]']             
                                )                                                                 
          

In [121]:
# Data loading classes
full = data_utils.DataLoaderFull()
input_full = full.get_training(validation_split=0.2, batch_size=64, shuffle=True)
valid_full = full.get_validation(validation_split=0.2, batch_size=64, shuffle=False)

Found 23501 files belonging to 3 classes.
Using 18801 files for training.
Found 23501 files belonging to 3 classes.
Using 4700 files for validation.


In [123]:
print(input_full.as_numpy_iterator().next()[0].shape)
test.output_shape

(64, 720, 1280, 3)


(None, 21, 38, 1536)